In [1]:
#import libraries
#Need to pip install selenium
from splinter import Browser
from bs4 import BeautifulSoup as soup
import re
import pandas as pd
import matplotlib.pyplot as plt
import time
from datetime import date
from selenium.webdriver.common.keys import Keys

In [2]:
abbreviation_to_name = {
    # https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States#States.
    "AK": "Alaska",
    "AL": "Alabama",
    "AR": "Arkansas",
    "AZ": "Arizona",
    "CA": "California",
    "CO": "Colorado",
    "CT": "Connecticut",
    "DE": "Delaware",
    "FL": "Florida",
    "GA": "Georgia",
    "HI": "Hawaii",
    "IA": "Iowa",
    "ID": "Idaho",
    "IL": "Illinois",
    "IN": "Indiana",
    "KS": "Kansas",
    "KY": "Kentucky",
    "LA": "Louisiana",
    "MA": "Massachusetts",
    "MD": "Maryland",
    "ME": "Maine",
    "MI": "Michigan",
    "MN": "Minnesota",
    "MO": "Missouri",
    "MS": "Mississippi",
    "MT": "Montana",
    "NC": "North Carolina",
    "ND": "North Dakota",
    "NE": "Nebraska",
    "NH": "New Hampshire",
    "NJ": "New Jersey",
    "NM": "New Mexico",
    "NV": "Nevada",
    "NY": "New York",
    "OH": "Ohio",
    "OK": "Oklahoma",
    "OR": "Oregon",
    "PA": "Pennsylvania",
    "RI": "Rhode Island",
    "SC": "South Carolina",
    "SD": "South Dakota",
    "TN": "Tennessee",
    "TX": "Texas",
    "UT": "Utah",
    "VA": "Virginia",
    "VT": "Vermont",
    "WA": "Washington",
    "WI": "Wisconsin",
    "WV": "West Virginia",
    "WY": "Wyoming",
    # https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States#Federal_district.
    "DC": "District of Columbia",
    # https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States#Inhabited_territories.
    "AS": "American Samoa",
    "GU": "Guam GU",
    "MP": "Northern Mariana Islands",
    "PR": "Puerto Rico PR",
    "VI": "U.S. Virgin Islands",
}

name_to_abbreviation = {v: k for k, v in abbreviation_to_name.items()}

In [3]:
# Set up splinter
browser = Browser('chrome')

# base url
base_url = "https://www.facebook.com/marketplace/sanfrancisco/search?"

# search parameters
# days_listed = 7
deliveryMethod = "local_pick_up"
query = "honda%20insight"
availability = "in%20stock"

#full url
url = f"{base_url}availability={availability}&deliveryMethod={deliveryMethod}&query={query}"

#City to Test
CITY = "Boulder, Colorado"

In [4]:
#visit site
browser.visit(url)

In [5]:
if browser.is_element_present_by_css('div[aria-label="Close"]', wait_time=10):
    # Click on the element once it's found
    browser.find_by_css('div[aria-label="Close"]').first.click()

In [6]:
browser.find_by_text(f'San Francisco, California').click()

In [7]:
browser.find_by_css('input[aria-label="Location"]').click()
while (len(browser.find_by_css('input[aria-label="Location"]').value)) > 0:
    browser.find_by_css('input[aria-label="Location"]').type(Keys.BACKSPACE)
browser.find_by_css('input[aria-label="Location"]').type(CITY)

'Boulder, Colorado'

In [8]:
browser.find_by_css('input[aria-label="Location"]').type(Keys.DOWN)
browser.find_by_css('input[aria-label="Location"]').type(Keys.ENTER)

'\ue007'

In [9]:
browser.find_by_css('div[aria-label="Apply"]').click()


In [10]:
# scroll down to load more results

scroll_count = 4

scroll_delay = 2

for _ in range(scroll_count):
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    time.sleep(scroll_delay)

In [6]:
# Parse HTML
html = browser.html

#create BS object from HTML
market_soup = soup(html, "html.parser")

In [524]:
# End browsing session
browser.quit()

In [7]:
card_div = market_soup.find_all('div', class_="x9f619 x78zum5 xdt5ytf x1qughib x1rdy4ex xz9dl7a xsag5q8 xh8yej3 xp0eagm x1nrcals")
titles_list = []
for title_div in card_div:
    title_exists = title_div.find('span', class_="x1lliihq x6ikm8r x10wlt62 x1n2onr6")
    if title_exists:
        titles_list.append(title_exists.text.strip())
    else:
        titles_list.append("No Title Provided")
        
titles_div = market_soup.find_all('span', class_="x1lliihq x6ikm8r x10wlt62 x1n2onr6")
print(len(card_div),len(titles_div))

24 26


In [10]:
# Extract all the info, put into lists
titles_div = market_soup.find_all('span', class_="x1lliihq x6ikm8r x10wlt62 x1n2onr6")
titles_list = []
for title in titles_div:
    if title.text.strip():
        titles_list.append(title.text.strip())
    else:
        titles_list.append('Title Not Provided')
titles_list.pop(0)
titles_list.pop(0)

prices_div = market_soup.find_all('span', class_="x193iq5w xeuugli x13faqbe x1vvkbs xlh3980 xvmahel x1n0sxbx x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x x4zkp8e x3x7a5m x1lkfr7t x1lbecb7 x1s688f xzsf02u")
prices_list = [price.text.strip() for price in prices_div]

miles_div = market_soup.find_all('span', class_="x1lliihq x6ikm8r x10wlt62 x1n2onr6 xlyipyv xuxw1ft")
miles_list = [mile.text.strip() for mile in miles_div]

image_elems = market_soup.find_all('img', class_= "xt7dq6l xl1xv1r x6ikm8r x10wlt62 xh8yej3")
image_list = [image.get('src') for image in image_elems]

urls_elems = market_soup.find_all('a', class_= "x1i10hfl xjbqb8w x1ejq31n xd10rxx x1sy0etr x17r0tee x972fbf xcfux6l x1qhh985 xm0m39n x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz x1heor9g x1sur9pj xkrqix3 x1lku1pv")
urls_list = [url.get('href') for url in urls_elems]




In [11]:
titles_list

['Honda Insight Hybrid 2014',
 '2010 Honda insight EX Hatchback 4D',
 '2012 Honda insight Hatchback 4D',
 '2019 Honda insight LX Sedan 4D',
 '2020 Honda insight *hybrid*',
 '2002 Honda insight Hybrid Manual',
 '2010 Honda insight LX Hatchback 4D',
 '2012 Honda insight hybrid',
 '2013 Honda insight',
 '2010 Honda insight LX Hatchback 4D',
 '2022 Honda insight',
 '2010 Honda insight LX Hatchback 4D',
 '2010 Honda insight',
 '2019 Honda insight Touring Sedan 4D',
 '2014 Honda insight Hatchback 4D',
 '2019 Honda insight LX Sedan 4D',
 '2019 Honda insight EX Sedan 4D',
 '2021 Honda insight EX Sedan 4D',
 '2010 Honda insight',
 '2022 Honda insight EX Sedan 4D',
 '2010 Honda insight LX Hatchback 4D',
 '2010 Honda insight LX Hatchback 4D',
 '2019 Honda insight EX Sedan 4D',
 '2012 Honda insight EX Hatchback 4D']

In [12]:
titles_div

[<span class="x1lliihq x6ikm8r x10wlt62 x1n2onr6" style="-webkit-box-orient: vertical; -webkit-line-clamp: 2; display: -webkit-box;">Filters</span>,
 <span class="x1lliihq x6ikm8r x10wlt62 x1n2onr6" style="-webkit-box-orient: vertical; -webkit-line-clamp: 2; display: -webkit-box;">Categories</span>,
 <span class="x1lliihq x6ikm8r x10wlt62 x1n2onr6" style="-webkit-box-orient: vertical; -webkit-line-clamp: 2; display: -webkit-box;">Honda Insight Hybrid 2014</span>,
 <span class="x1lliihq x6ikm8r x10wlt62 x1n2onr6" style="-webkit-box-orient: vertical; -webkit-line-clamp: 2; display: -webkit-box;">2010 Honda insight EX Hatchback 4D</span>,
 <span class="x1lliihq x6ikm8r x10wlt62 x1n2onr6" style="-webkit-box-orient: vertical; -webkit-line-clamp: 2; display: -webkit-box;">2012 Honda insight Hatchback 4D</span>,
 <span class="x1lliihq x6ikm8r x10wlt62 x1n2onr6" style="-webkit-box-orient: vertical; -webkit-line-clamp: 2; display: -webkit-box;">2019 Honda insight LX Sedan 4D</span>,
 <span clas

In [13]:
image_list

['https://scontent-lax3-2.xx.fbcdn.net/v/t45.5328-4/442480886_453430647095576_951752722786574586_n.jpg?stp=c0.320.526.526a_dst-jpg_p526x395&_nc_cat=111&ccb=1-7&_nc_sid=247b10&_nc_ohc=xqBMdoS8FAcQ7kNvgH4I9o7&_nc_ht=scontent-lax3-2.xx&oh=00_AYACEUqxOwt1cvt716BmZMtpTlo73_DhvAn9nNOje4Rd8Q&oe=664AC93E',
 'https://scontent-lax3-2.xx.fbcdn.net/v/t39.30808-6/441542338_422509993886326_2069947625460445571_n.jpg?stp=c0.58.526.526a_dst-jpg_p526x296&_nc_cat=103&ccb=1-7&_nc_sid=5f2048&_nc_ohc=_opLffcrosUQ7kNvgGpIE3X&_nc_ht=scontent-lax3-2.xx&oh=00_AYCeKD8JEVg0miIJdiIg0GYRrDtzUvR1t8wjrf-I0nzLvw&oe=664AFDCA',
 'https://scontent-lax3-2.xx.fbcdn.net/v/t39.30808-6/441400819_7873761382682295_6435062101287261820_n.jpg?stp=c90.0.540.540a_dst-jpg_p180x540&_nc_cat=107&ccb=1-7&_nc_sid=5f2048&_nc_ohc=olF3yDPWAmoQ7kNvgFXX0K0&_nc_ht=scontent-lax3-2.xx&oh=00_AYCKCxird73LYZM8dP5uJFkWpC8yXgAXteGD_haOgrKtrQ&oe=664AD82C',
 'https://scontent-lax3-2.xx.fbcdn.net/v/t39.30808-6/441951710_7755662981145210_13291501456491470

In [14]:
miles_list

['Log In',
 'Log In',
 'Notify Me',
 'Create new listing',
 'Sort by',
 'Delivery method: Local pickup',
 'Condition',
 'Date listed',
 'Availability',
 'San Francisco · 40 mi',
 'Filters',
 'Notify Me',
 'Sacramento, CA',
 'Redwood City, CA',
 '120K miles',
 'West Sacramento, CA',
 '162K miles',
 'San Francisco, CA',
 '43K miles',
 'Carmichael, CA',
 '70K miles',
 'San Francisco, CA',
 '160K miles',
 'Petaluma, CA',
 '215K miles',
 'Hayward, CA',
 '206K miles',
 'Mountain View, CA',
 '192K miles',
 'San Mateo, CA',
 '268K miles',
 'Citrus Heights, CA',
 '22K miles',
 'Pleasant Hill, CA',
 '242K miles',
 'San Jose, CA',
 '169K miles · Dealership',
 'San Jose, CA',
 '90K miles',
 'Sacramento, CA',
 '220K miles',
 'Sausalito, CA',
 '170K miles',
 'Sacramento, CA',
 '120K miles',
 'Rancho Cordova, CA',
 '50K miles',
 'Salinas, CA',
 '113K miles',
 'Sacramento, CA',
 '61K miles',
 'Sacramento, CA',
 '179K miles',
 'San Francisco, CA',
 '185K miles',
 'Daly City, CA',
 '50K miles',
 'Daly C

In [15]:
urls_list

['/marketplace/item/458329213262304/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
 '/marketplace/item/457678463315097/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
 '/marketplace/item/416078744542265/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
 '/marketplace/item/3717544678503326/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
 '/marketplace/item/1178685260246676/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
 '/marketplace/item/1038589247401187/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
 '/marketplace/item/726291939237098/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
 '/marketplace/item/1119586016049340/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
 '/marketplace/item/437465502221321/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
 '/marketplace/item/1147374986405

In [849]:
prices_list

['$2,500',
 '$800',
 '$5,700',
 '$1,700',
 '$5,800',
 '$1,600',
 '$33,000',
 '$9,995',
 '$7,500',
 '$22,850',
 '$1,200',
 '$17,400',
 '$6,900',
 '$1,950',
 '$3,500',
 '$999',
 '$3,800',
 '$100',
 '$6,000',
 '$2,500',
 '$22,201',
 '$5,900',
 '$8,000',
 '$7,000',
 '$2,000',
 '$2,000',
 '$3,800',
 '$850',
 '$9,999',
 '$4,799',
 '$6,500',
 '$3,500',
 '$3,885',
 '$2,800',
 '$3,500',
 '$15',
 '$4,700',
 '$15,750',
 '$500',
 '$14,900',
 '$8,900',
 '$475',
 '$6,500',
 '$900',
 '$600',
 '$16,999',
 '$2,500',
 '$12,889',
 '$2,300',
 '$20',
 '$14,995',
 '$1,900',
 '$1',
 '$10,500',
 '$3,500',
 '$3,500',
 '$3,500',
 '$10,000',
 '$1,200',
 '$5,000',
 '$2,424',
 '$13,000',
 '$1',
 '$1,100',
 '$1,200',
 '$1,000',
 '$2,424',
 '$1,200',
 '$11,950',
 '$26,990',
 '$3,500',
 '$12,999',
 '$45',
 '$15,995',
 '$21,000',
 '$2,600',
 '$6,990',
 '$5,900',
 '$12,500',
 '$24,985',
 '$15,990',
 '$19,988',
 '$17,000',
 '$23,000',
 '$20,900',
 '$22,785',
 '$45',
 '$16,490',
 '$10,998',
 '$420',
 '$1,000',
 '$1,600',

In [850]:
miles_list

['Log In',
 'Log In',
 'Notify Me',
 'Create new listing',
 'Sort by',
 'Delivery method: Local pickup',
 'Condition',
 'Date listed',
 'Availability',
 'Boulder · 40 mi',
 'Filters',
 'Notify Me',
 'Commerce City, CO',
 '165K miles',
 'Fort Collins, CO',
 '153K miles',
 'Aurora, CO',
 '1.4K miles',
 'Englewood, CO',
 '180K miles',
 'Arvada, CO',
 '160K miles',
 'Denver, CO',
 '',
 'Denver, CO',
 '26K miles',
 'Englewood, CO',
 '123K miles',
 'Englewood, CO',
 '94K miles',
 'Denver, CO',
 '14K miles · Dealership',
 'Aurora, CO',
 '172K miles',
 'Englewood, CO',
 '46K miles',
 'Denver, CO',
 '185K miles',
 'Colorado Springs, CO',
 '250K miles · Dealership',
 'Englewood, CO',
 '147K miles · Dealership',
 'Aurora, CO',
 '81K miles',
 'Denver, CO',
 '200K miles',
 'Denver, CO',
 '150K miles',
 'Denver, CO',
 '187K miles',
 'Aurora, CO',
 '115K miles',
 'Colorado Springs, CO',
 '62K miles',
 'Albuquerque, NM',
 '109K miles',
 'Denver, CO',
 '276K miles',
 'Denver, CO',
 'Colorado Springs, C

In [851]:
# pattern for City, State (ex: Los Angeles, CA)
location_pattern = re.compile(r"[a-zA-Z -']+[,]\s[A-Z]")
# pattern for miles (ex: 100K miles)
miles_pattern = re.compile(r'([0-9.]+)[K]? miles')
# regex pattern for the item after the last valid piece of mileage data.
login_or_sign = re.compile(r'Log in or sign up for Facebook')

filler_city = CITY.split(', ')[0]
filler_state = CITY.split(', ')[1]

# initialize empty list
miles_list2 = []

#iterate through the original mileage entries
for item in miles_list:

    # Current item is location, previous item is location (2 locations in a row)
    if location_pattern.match(item) and len(miles_list2) >= 1 and location_pattern.match(miles_list2[-1]):
        miles_list2.append('0K miles')
        miles_list2.append(item)

    # Current item is empty string, Previous item is location (Miles is empty string)
    elif len(item) == 0 and location_pattern.match(miles_list2[-1]) and len(miles_list2) >=1:
        miles_list2.append('0K miles')

    # Previous item is miles, and current item is miles (Missing Location)
    elif miles_pattern.match(item) and len(miles_list2) >= 1 and miles_pattern.match(miles_list2[-1]):
        miles_list2.append(f'{filler_city}, {filler_state}')
        miles_list2.append(item)

    # Previous item is location, and current item does not match (2 locations in a row)
    elif len(miles_list2) >= 1 and location_pattern.match(miles_list2[-1]) and miles_pattern.match(item) == None:
        miles_list2.append('0K miles')

    # Last listing does not have mileage, so append 0K
    elif len(miles_list2) >= 1 and login_or_sign.match(item) and location_pattern.match(miles_list2[-1]):
        miles_list2.append('0k miles')


    else:
        miles_list2.append(item)



In [852]:
miles_list2

['Log In',
 'Log In',
 'Notify Me',
 'Create new listing',
 'Sort by',
 'Delivery method: Local pickup',
 'Condition',
 'Date listed',
 'Availability',
 'Boulder · 40 mi',
 'Filters',
 'Notify Me',
 'Commerce City, CO',
 '165K miles',
 'Fort Collins, CO',
 '153K miles',
 'Aurora, CO',
 '1.4K miles',
 'Englewood, CO',
 '180K miles',
 'Arvada, CO',
 '160K miles',
 'Denver, CO',
 '0K miles',
 'Denver, CO',
 '26K miles',
 'Englewood, CO',
 '123K miles',
 'Englewood, CO',
 '94K miles',
 'Denver, CO',
 '14K miles · Dealership',
 'Aurora, CO',
 '172K miles',
 'Englewood, CO',
 '46K miles',
 'Denver, CO',
 '185K miles',
 'Colorado Springs, CO',
 '250K miles · Dealership',
 'Englewood, CO',
 '147K miles · Dealership',
 'Aurora, CO',
 '81K miles',
 'Denver, CO',
 '200K miles',
 'Denver, CO',
 '150K miles',
 'Denver, CO',
 '187K miles',
 'Aurora, CO',
 '115K miles',
 'Colorado Springs, CO',
 '62K miles',
 'Albuquerque, NM',
 '109K miles',
 'Denver, CO',
 '276K miles',
 'Denver, CO',
 '0K miles',


In [853]:
# Clean Location and Miles Data
miles_pattern_miles = r'([0-9.]+)[K]? miles'
miles_pattern_km = r'(\d+)K km'
location_pattern = r'[a-zA-Z ]+[,]\s[A-Z][A-Z]'

#Edge Cases
full_state_pattern_one = r'[a-zA-Z ]+[,]\s[A-Z][a-z]+'
full_state_pattern_two = r'[a-zA-Z ]+[,]\s[A-Z][a-z]+\s[A-Z][a-z]+'


miles_clean = []
locations_clean =[]

for item in miles_list2:

    match_miles_km = re.search(miles_pattern_km, item)

    match_mileage_miles = re.search(miles_pattern_miles, item)

    match_location = re.search(location_pattern, item)

    match_full_state_one = re.search(full_state_pattern_one, item)
    
    match_full_state_two = re.search(full_state_pattern_two, item)


    if match_miles_km or match_mileage_miles or match_location or match_full_state_one or match_full_state_two:
        if match_miles_km:
            miles_clean.append(int(match_miles_km.group(1)) *1000)

        if match_mileage_miles:
            miles_clean.append(int(float(match_mileage_miles.group(1))) *1000)
            # print(match_mileage_miles)
            # print(int(float(match_mileage_miles.group(1))) *1000)

        if match_location:
            locations_clean.append(item)
            print(match_location)
            print(item)

        if match_full_state_one or match_full_state_two:
            locations_clean.append(name_to_abbreviation[item])

    else:
        print('NON-MATCHING MILES/LOCATION', item)

NON-MATCHING MILES/LOCATION Log In
NON-MATCHING MILES/LOCATION Log In
NON-MATCHING MILES/LOCATION Notify Me
NON-MATCHING MILES/LOCATION Create new listing
NON-MATCHING MILES/LOCATION Sort by
NON-MATCHING MILES/LOCATION Delivery method: Local pickup
NON-MATCHING MILES/LOCATION Condition
NON-MATCHING MILES/LOCATION Date listed
NON-MATCHING MILES/LOCATION Availability
NON-MATCHING MILES/LOCATION Boulder · 40 mi
NON-MATCHING MILES/LOCATION Filters
NON-MATCHING MILES/LOCATION Notify Me
<re.Match object; span=(0, 17), match='Commerce City, CO'>
Commerce City, CO
<re.Match object; span=(0, 16), match='Fort Collins, CO'>
Fort Collins, CO
<re.Match object; span=(0, 10), match='Aurora, CO'>
Aurora, CO
<re.Match object; span=(0, 13), match='Englewood, CO'>
Englewood, CO
<re.Match object; span=(0, 10), match='Arvada, CO'>
Arvada, CO
<re.Match object; span=(0, 10), match='Denver, CO'>
Denver, CO
<re.Match object; span=(0, 10), match='Denver, CO'>
Denver, CO
<re.Match object; span=(0, 13), match='En

In [854]:
len(locations_clean)

120

In [855]:
locations_clean

['Commerce City, CO',
 'Fort Collins, CO',
 'Aurora, CO',
 'Englewood, CO',
 'Arvada, CO',
 'Denver, CO',
 'Denver, CO',
 'Englewood, CO',
 'Englewood, CO',
 'Denver, CO',
 'Aurora, CO',
 'Englewood, CO',
 'Denver, CO',
 'Colorado Springs, CO',
 'Englewood, CO',
 'Aurora, CO',
 'Denver, CO',
 'Denver, CO',
 'Denver, CO',
 'Aurora, CO',
 'Colorado Springs, CO',
 'Albuquerque, NM',
 'Denver, CO',
 'Denver, CO',
 'Colorado Springs, CO',
 'Paonia, CO',
 'Longmont, CO',
 'Loveland, CO',
 'Evanston, WY',
 'Denver, CO',
 'Albuquerque, NM',
 'Colorado Springs, CO',
 'Boulder, CO',
 'Aurora, CO',
 'Denver, CO',
 'Denver, CO',
 'Denver, CO',
 'Aurora, CO',
 'Bayard, NE',
 'Denver, CO',
 'Aurora, CO',
 'Colorado Springs, CO',
 'Fort Collins, CO',
 'Aurora, CO',
 'Pueblo, CO',
 'Englewood, CO',
 'Aurora, CO',
 'Fort Collins, CO',
 'North Platte, NE',
 'Denver, CO',
 'Denver, CO',
 'Brighton, CO',
 'Aurora, CO',
 'Denver, CO',
 'Denver, CO',
 'Aurora, CO',
 'Denver, CO',
 'Aurora, CO',
 'Golden, CO

In [856]:
len(miles_clean)

120

In [811]:
# Make Prices into Integer
prices_clean = []

for price in prices_list:
    if price == 'Free':
        prices_clean.append(0)
    elif price == "·":
        prices_clean.append(0)
    else:
        prices_clean.append(int(re.sub(r'[₹,M,X,$,]','', price)))


In [812]:
# Make URLS full url
urls_clean = []

for url in urls_list:
    urls_clean.append('https://www.facebook.com' + url)


In [818]:
len(titles_list)

119

In [701]:
# add all values to a list of dictionaries
vehicles_list = []

for i, item in enumerate(titles_list):
    cars_dict = {}
    first_gen = False
    insight = False
    parts = False

    year = 0
    year_pattern = r'[0-9]{4}'
    year_match = re.search(year_pattern,titles_list[i])
    first_gen_years = [2000, 2001, 2002, 2003, 2004, 2005, 2006]

    #Checks for year
    if year_match:
        year = int(year_match[0])
        print(year_match)
        print(year)

    #Checks if insight is actually in the title
    if "insight" in titles_list[i].lower():
        insight = True

    #Checks if any cars are 2000 - 2006
    if year_match:
        if any(x == int(year_match[0]) for x in first_gen_years) and insight:
            print("first gen exists")
            first_gen = True

    #Checks if car listing is for parts
    if "part" in titles_list[i].lower():
        parts = True


    #Splits up the City and State from location
    city = locations_clean[i].split(', ')[0]
    state = locations_clean[i].split(', ')[-1]



    # Map out key value pairs
    cars_dict["date"] = date.today()
    cars_dict["title"] = titles_list[i]
    cars_dict["price"] = prices_clean[i]
    cars_dict["city"] = city
    cars_dict["state"] = state
    cars_dict["mileage"] = miles_clean[i]
    cars_dict["url"] = urls_clean[i]
    cars_dict["image"] = image_list[i]
    cars_dict["insight"] = insight
    cars_dict["first_gen"] = first_gen
    cars_dict["parts"] = parts
    cars_dict["site"] = "fb"
    cars_dict["year"] = year
    vehicles_list.append(cars_dict)


<re.Match object; span=(0, 4), match='2002'>
2002
first gen exists
<re.Match object; span=(0, 4), match='2019'>
2019
<re.Match object; span=(0, 4), match='2010'>
2010
<re.Match object; span=(0, 4), match='2011'>
2011
<re.Match object; span=(0, 4), match='2012'>
2012
<re.Match object; span=(0, 4), match='2019'>
2019
<re.Match object; span=(0, 4), match='2019'>
2019
<re.Match object; span=(0, 4), match='2010'>
2010
<re.Match object; span=(0, 4), match='2010'>
2010
<re.Match object; span=(0, 4), match='2010'>
2010
<re.Match object; span=(0, 4), match='2010'>
2010
<re.Match object; span=(0, 4), match='2021'>
2021
<re.Match object; span=(0, 4), match='2014'>
2014
<re.Match object; span=(0, 4), match='2012'>
2012
<re.Match object; span=(0, 4), match='2019'>
2019
<re.Match object; span=(0, 4), match='2019'>
2019
<re.Match object; span=(0, 4), match='2010'>
2010
<re.Match object; span=(0, 4), match='2011'>
2011
<re.Match object; span=(0, 4), match='2018'>
2018
<re.Match object; span=(0, 4), ma

In [702]:
vehicles_list

[{'date': datetime.date(2024, 4, 10),
  'title': '2002 Honda insight Hybrid Hatchback 2D',
  'price': 3650,
  'city': 'San Francisco',
  'state': 'CA',
  'mileage': 165000,
  'url': 'https://www.facebook.com/marketplace/item/1492382191345101/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
  'image': 'https://scontent-lax3-2.xx.fbcdn.net/v/t39.30808-6/433140773_122115755444240073_5556977948482205665_n.jpg?stp=c0.86.828.828a_dst-jpg_s261x260&_nc_cat=100&ccb=1-7&_nc_sid=5f2048&_nc_ohc=oA_kO9bkxMkAb5PrWPc&_nc_ht=scontent-lax3-2.xx&oh=00_AfABWDBO_l8RT8b3j3enQppB3LgSjkA8TZKOo6JxOXaNNQ&oe=661CE5CF',
  'insight': True,
  'first_gen': True,
  'parts': False,
  'site': 'fb',
  'year': 2002},
 {'date': datetime.date(2024, 4, 10),
  'title': '2019 Honda insight EX Sedan 4D',
  'price': 11000,
  'city': 'Denair',
  'state': 'CA',
  'mileage': 150000,
  'url': 'https://www.facebook.com/marketplace/item/1826502207823881/?ref=search&referral_code=null&referral_story_type=post&__

In [703]:
vehicles_df = pd.DataFrame(vehicles_list)

In [704]:
vehicles_df

,date,title,price,city,state,mileage,url,image,insight,first_gen,parts,site,year
0,2024-04-10,2002 Honda insight Hybrid Hatchback 2D,3650,San Francisco,CA,165000,https://www.facebook.com/marketplace/item/1492...,https://scontent-lax3-2.xx.fbcdn.net/v/t39.308...,True,True,False,fb,2002
1,2024-04-10,2019 Honda insight EX Sedan 4D,11000,Denair,CA,150000,https://www.facebook.com/marketplace/item/1826...,https://scontent-lax3-1.xx.fbcdn.net/v/t39.308...,True,False,False,fb,2019
2,2024-04-10,2010 Honda insight EX Hatchback 4D,1500,Vallejo,CA,125000,https://www.facebook.com/marketplace/item/1886...,https://scontent-lax3-1.xx.fbcdn.net/v/t39.308...,True,False,False,fb,2010
3,2024-04-10,2011 Honda insight Hybrid,6000,Fresno,CA,98000,https://www.facebook.com/marketplace/item/3823...,https://scontent-lax3-2.xx.fbcdn.net/v/t39.308...,True,False,False,fb,2011
4,2024-04-10,2012 Honda insight,6000,Roseville,CA,134000,https://www.facebook.com/marketplace/item/2519...,https://scontent-lax3-2.xx.fbcdn.net/v/t39.308...,True,False,False,fb,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,2024-04-10,Parting Out 2010 Honda Insight,100,Rancho Cordova,CA,0,https://www.facebook.com/marketplace/item/1877...,https://scontent-lax3-1.xx.fbcdn.net/v/t45.532...,True,False,True,fb,2010
92,2024-04-10,2019 Honda civic EX Hatchback 4D,1,Rancho Cordova,CA,63000,https://www.facebook.com/marketplace/item/4267...,https://scontent-lax3-2.xx.fbcdn.net/v/t39.308...,False,False,False,fb,2019
93,2024-04-10,2009 Toyota prius,12500,Richmond,CA,180000,https://www.facebook.com/marketplace/item/2923...,https://scontent-lax3-1.xx.fbcdn.net/v/t45.532...,False,False,False,fb,2009
94,2024-04-10,2007 Honda civic Hybrid Sedan 4D,900,Sonoma,CA,190000,https://www.facebook.com/marketplace/item/7797...,https://scontent-lax3-1.xx.fbcdn.net/v/t39.308...,False,False,False,fb,2007


In [705]:
filtered_df = vehicles_df[vehicles_df['insight']==True]

In [706]:
filtered_df

,date,title,price,city,state,mileage,url,image,insight,first_gen,parts,site,year
0,2024-04-10,2002 Honda insight Hybrid Hatchback 2D,3650,San Francisco,CA,165000,https://www.facebook.com/marketplace/item/1492...,https://scontent-lax3-2.xx.fbcdn.net/v/t39.308...,True,True,False,fb,2002
1,2024-04-10,2019 Honda insight EX Sedan 4D,11000,Denair,CA,150000,https://www.facebook.com/marketplace/item/1826...,https://scontent-lax3-1.xx.fbcdn.net/v/t39.308...,True,False,False,fb,2019
2,2024-04-10,2010 Honda insight EX Hatchback 4D,1500,Vallejo,CA,125000,https://www.facebook.com/marketplace/item/1886...,https://scontent-lax3-1.xx.fbcdn.net/v/t39.308...,True,False,False,fb,2010
3,2024-04-10,2011 Honda insight Hybrid,6000,Fresno,CA,98000,https://www.facebook.com/marketplace/item/3823...,https://scontent-lax3-2.xx.fbcdn.net/v/t39.308...,True,False,False,fb,2011
4,2024-04-10,2012 Honda insight,6000,Roseville,CA,134000,https://www.facebook.com/marketplace/item/2519...,https://scontent-lax3-2.xx.fbcdn.net/v/t39.308...,True,False,False,fb,2012
5,2024-04-10,2019 Honda insight Touring Sedan 4D,2500,San Jose,CA,90000,https://www.facebook.com/marketplace/item/9730...,https://scontent-lax3-1.xx.fbcdn.net/v/t39.308...,True,False,False,fb,2019
6,2024-04-10,2019 Honda insight EX Sedan 4D,13500,Sacramento,CA,120000,https://www.facebook.com/marketplace/item/1419...,https://scontent-lax3-1.xx.fbcdn.net/v/t39.308...,True,False,False,fb,2019
7,2024-04-10,2010 Honda insight EX Hatchback 4D,4500,Burlingame,CA,230000,https://www.facebook.com/marketplace/item/1067...,https://scontent-lax3-1.xx.fbcdn.net/v/t39.308...,True,False,False,fb,2010
8,2024-04-10,2010 Honda insight LX Hatchback 4D,6200,San Francisco,CA,184000,https://www.facebook.com/marketplace/item/4253...,https://scontent-lax3-1.xx.fbcdn.net/v/t39.308...,True,False,False,fb,2010
9,2024-04-10,2010 Honda insight EX,Pending,Sacramento,CA,140000,https://www.facebook.com/marketplace/item/1168...,https://scontent-lax3-1.xx.fbcdn.net/v/t39.308...,True,False,False,fb,2010


In [328]:
csv_file_path = f'/Users/evanishibashi/Projects/insight/csv/fb/{date.today()}.csv'

filtered_df.to_csv(csv_file_path, index=False)